# Tools

Tools: Enables agents to execute specific actions in external systems.
This component provides the capability to make API calls, database updates, file operations, and other practical actions.

## Tool use in Groq

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [17]:
from groq import Groq
import json
import requests

client = Groq(api_key=os.getenv("GROQ_API_KEY"))
MODEL = "meta-llama/llama-4-scout-17b-16e-instruct"

def calculate(expression: str):
    """Evaluate a mathematical expression"""
    try:
        # Attempt to evaluate the expression
        result =  eval(expression)
        return json.dumps({"result": result})
    except Exception as e:
        # Return an error message if the expression is invalid
        return json.dumps({"error": "Invalid Expression"})

def get_weather(latitude: str, longitude: str):
    """Get current temperature for provided coordinates in celsius."""
    response = requests.get("https://api.open-meteo.com/v1/forecast?latitude="+latitude+"&longitude="+longitude+"&current=temperature_2m,wind_speed_10m")
    
    data = response.json()
    temperature = data["current"]["temperature_2m"]
    print(f"Temperature: {temperature}°C")
    # Return as JSON string to match the calculate function format
    return json.dumps({"temperature": temperature, "unit": "celsius"})

In [56]:
def run_conversation(user_prompt: str):
    messages = [
        {
            "role": "system",
            "content": "You are a helpful agent. You are equiped with some tools use them according to the need."
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]
    
    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate",
                "description": "Evaluate a mathematical expression",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "The mathematical expression to evaluate." 
                        }
                    },
                    "required": ["expression"],
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Get current temperature for provided coordinates in celsius.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "latitude": {
                            "type": "string"
                        },
                        "longitude": {
                            "type": "string"
                        }
                    },
                    "required": ["latitude", "longitude"]
                },
                "strict": True,
            }
        }
    ]
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools = tools,
        tool_choice="auto",
        max_completion_tokens=4096,
    )
    
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls:
        available_functions = {
            "calculate": calculate,
            "get_weather": get_weather,
        }
        messages.append(response_message)
        
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            
            function_response = function_to_call(**function_args)
            # function_response = function_to_call(
            #     expression=function_args.get("expression")
            # )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response
                }
            )
        second_response = client.chat.completions.create(
            model = MODEL,
            messages=messages
        )
        print(messages)
        print("="*60)  
        return second_response.choices[0].message.content
    
    
user_prompt = "what is the temparature of kanpur and after that do this calculation 30*4+100-20"
print(run_conversation(user_prompt=user_prompt)) 



Temperature: 26.1°C
[{'role': 'system', 'content': 'You are a helpful agent. You are equiped with some tools use them according to the need.'}, {'role': 'user', 'content': 'what is the temparature of kanpur and after that do this calculation 30*4+100-20'}, ChatCompletionMessage(content="To proceed, I need to get the temperature of Kanpur. Kanpur's coordinates are approximately 26.4499° N, 80.3319° E.\n\nFor the temperature: ", role='assistant', executed_tools=None, function_call=None, reasoning=None, tool_calls=[ChatCompletionMessageToolCall(id='nc3qh00qw', function=Function(arguments='{"latitude":"26.4499","longitude":"80.3319"}', name='get_weather'), type='function')]), {'tool_call_id': 'nc3qh00qw', 'role': 'tool', 'name': 'get_weather', 'content': '{"temperature": 26.1, "unit": "celsius"}'}]
The current temperature in Kanpur is 26.1°C.

Now, let's perform the calculation:

30 * 4 = 120
120 + 100 = 220
220 - 20 = 200

The result of the calculation is 200.


### Parallel tool use

In [63]:
def get_temperature(latitude: str, longitude: str):
    response = requests.get("https://api.open-meteo.com/v1/forecast?latitude="+latitude+"&longitude="+longitude+"&current=temperature_2m")
    data = response.json()
    temperature = data['current']['temperature_2m']
    return json.dumps({"temperature": temperature, "unit": "celsius"})

def get_relative_humidity(latitude: str, longitude: str):
    response = requests.get("https://api.open-meteo.com/v1/forecast?latitude="+latitude+"&longitude="+longitude+"&current=relative_humidity_2m")
    data = response.json()
    humidity = data['current']['relative_humidity_2m']
    return json.dumps({"relative_humidity": humidity, "unit": "%"})



def run_convo(user_prompt):
    messages = [
        {
            "role": "system",
            "content": "You are a helpful agent. You are equiped with some tools, use them as needed."
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_temperature",
                "description": "Get current temperature for provided coordinates in celsius.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "latitude": {"type": "string"},
                        "longitude": {"type": "string"},
                    },
                    "required": ["latitude", "longitude"],
                },
                "strict": True,
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_relative_humidity",
                "description": "Get current relative humidity for provided coordinates in percentage.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "latitude": {"type": "string"},
                        "longitude": {"type": "string"},
                    },
                    "required": ["latitude", "longitude"],
                },
                "strict": True,
            }
        }
    ]
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_completion_tokens=4096,
    )
    
    response_message = response.choices[0].message
    
    tool_calls = response_message.tool_calls
    messages.append(response_message)
    available_functions = {
        "get_temperature": get_temperature,
        "get_relative_humidity": get_relative_humidity,
    }
    # print(tool_calls)
    
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(**function_args)
        
        messages.append(
            {
                "role": "tool",
                "content": str(function_response),
                "tool_call_id": tool_call.id
            }
        )
        
        # print(messages)
        # print("="*100)

    final_response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools, 
        tool_choice="auto", 
        max_completion_tokens=4096
    )
    
    print(final_response.choices[0].message.content)
    
user_prompt = "What's the temperature and relative humidity like in New York and London?"
run_convo(user_prompt)


The current temperature in New York is 24.1°C and the relative humidity is 77%. 

The current temperature in London is 20.1°C and the relative humidity is 83%.


## Tool use groq and gemini langchain

In [91]:
from langchain_core.tools import tool

@tool
def add(a: str, b: str) -> int:
    """Add two integers a and b."""
    return int(a) + int(b)

@tool
def multiply(a: str, b: str) -> int:
    """Multiply two integers a and b."""
    return int(a) * int(b)

tools = [add, multiply]

In [93]:
from pydantic import BaseModel, Field


# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class Add(BaseModel):
    """Add two integers together."""

    a: str = Field(..., description="First integer")
    b: str = Field(..., description="Second integer")


class Multiply(BaseModel):
    """Multiply two integers together."""

    a: str = Field(..., description="First integer")
    b: str = Field(..., description="Second integer")


tools = [Add, Multiply]

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(model=MODEL, model_provider="groq")
# llm = init_chat_model(model='gemini-2.0-flash', model_provider="google_genai")

In [94]:
llm_with_tools = llm.bind_tools(tools)

In [90]:
query = "what is 10+2? Also, what is 11*9?"
llm_with_tools.invoke(query).tool_calls

[{'name': 'Add',
  'args': {'a': '10', 'b': '2'},
  'id': '7ehj757ft',
  'type': 'tool_call'},
 {'name': 'Multiply',
  'args': {'a': '11', 'b': '9'},
  'id': 'sd74xgqb9',
  'type': 'tool_call'}]

In [88]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

chain = llm_with_tools | PydanticToolsParser(tools=[Add, Multiply])
chain.invoke(query)

[Add(a='10', b='2'), Multiply(a='11', b='9')]

In [99]:
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage

def run_conversation(human_query):
    system_prompt = """You are bad at math but are an expert at using a calculator. 

Use past tool usage as an example of how to correctly use the tools."""
    messages = [SystemMessage(system_prompt), HumanMessage(human_query)]
    ai_messages = llm_with_tools.invoke(messages)
    messages.append(ai_messages)
    available_tools = {
        "add": add,
        "multiply": multiply,
    }
    for tool_call in ai_messages.tool_calls:
        tool_name = tool_call["name"]
        selected_tool = available_tools[tool_name.lower()]
        tool_output = selected_tool(tool_call["args"])
        messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
    
    print(messages)
    final_messages = llm_with_tools.invoke(messages)
    print("="*100)
    print(final_messages.content)

run_conversation(query)    

[SystemMessage(content='You are bad at math but are an expert at using a calculator. \n\nUse past tool usage as an example of how to correctly use the tools.', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is 10+2? Also, what is 11*9?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'nmee04g9e', 'function': {'arguments': '{"a":"10","b":"2"}', 'name': 'Add'}, 'type': 'function'}, {'id': 'jck6haffb', 'function': {'arguments': '{"a":"11","b":"9"}', 'name': 'Multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 776, 'total_tokens': 829, 'completion_time': 0.123664831, 'prompt_time': 0.022950529, 'queue_time': 0.046204550999999996, 'total_time': 0.14661536}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_79da0e0073', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b3f116a

In [101]:
run_conversation("what is 119 time 8 plus 20?")

[SystemMessage(content='You are bad at math but are an expert at using a calculator. \n\nUse past tool usage as an example of how to correctly use the tools.', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is 119 time 8 plus 20?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '8fpp2445z', 'function': {'arguments': '{"a":"119","b":"8"}', 'name': 'Multiply'}, 'type': 'function'}, {'id': 'wbjnpsfbg', 'function': {'arguments': '{"a":"952","b":"20"}', 'name': 'Add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 770, 'total_tokens': 862, 'completion_time': 0.209401243, 'prompt_time': 0.022741704, 'queue_time': 0.04428341599999999, 'total_time': 0.232142947}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ff2b96d9-f

## Gemini Function Calling

In [102]:
def set_light_values(brightness: int, color_temp: int) -> dict[str, int | str]:
    """Set the brightness and color temperature of a room light. (mock API)
    
    Args: 
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness.
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.
        
    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {"brightness": brightness, "colorTemperature": color_temp}


In [121]:
from google import genai
from google.genai import types
import base64

client = genai.Client(
    api_key=os.getenv("GEMINI_API_KEY"),
)

function_declarations = [
    {
        "name": "set_light_values",
        "description": "Set the brightness and color temperature of a room light",
        "parameters": {
            "type": "object",
            "properties": {
                "brightness": {
                    "type": "integer",
                    "description": "Light level from 0 to 100. Zero is off and 100 is full brightness"
                },
                "color_temp": {
                    "type": "string",
                    "enum": ["daylight", "cool", "warm"],
                    "description": "Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`"
                }
            },
            "required": ["brightness", "color_temp"],
        }
    }
]

tools = types.Tool(function_declarations=function_declarations)
config = types.GenerateContentConfig(
    tools=[tools],
    thinking_config= types.ThinkingConfig(
        include_thoughts=True,
    )
)


def run_conversation(human_prompt):
    content = [
        types.Content(
            role="user", parts=[types.Part(text=human_prompt)]
        )
    ]
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=content,
        config=config
    )
    
    tool_call = response.candidates[0].content.parts[0].function_call
    
    if tool_call.name == "set_light_values":
        result = set_light_values(**tool_call.args)
        print(f"Function execution result: {result}")
    
    function_response_part = types.Part.from_function_response(
        name = tool_call.name,
        response = {"result": result}
    )
    
    content.append(response.candidates[0].content)
    thought_signature = response.candidates[0].content.parts[0].thought_signature
    if thought_signature:
        print(f"Thought Signature: {base64.b64encode(thought_signature).decode("utf-8")}")
    content.append(types.Content(role="tool", parts=[function_response_part]))
    
    final_response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents= content,
        config=config
    )
    print(content)
    print("="*80)
    print(final_response.text)

run_conversation("Turn the lights down to a romantic level.")
    

Function execution result: {'brightness': 20, 'colorTemperature': 'warm'}
[Content(
  parts=[
    Part(
      text='Turn the lights down to a romantic level.'
    ),
  ],
  role='user'
), Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'brightness': 20,
          'color_temp': 'warm'
        },
        name='set_light_values'
      )
    ),
  ],
  role='model'
), Content(
  parts=[
    Part(
      function_response=FunctionResponse(
        name='set_light_values',
        response={
          'result': {
            'brightness': 20,
            'colorTemperature': 'warm'
          }
        }
      )
    ),
  ],
  role='tool'
)]
OK. I've set the lights to a brightness of 20 and a warm color temperature.



### Parallel function calling

In [122]:
def power_disco_ball_impl(power: bool) -> dict:
    """Powers the spinning disco ball.

    Args:
        power: Whether to turn the disco ball on or off.

    Returns:
        A status dictionary indicating the current state.
    """
    return {"status": f"Disco ball powered {'on' if power else 'off'}"}

def start_music_impl(energetic: bool, loud: bool) -> dict:
    """Play some music matching the specified parameters.

    Args:
        energetic: Whether the music is energetic or not.
        loud: Whether the music is loud or not.

    Returns:
        A dictionary containing the music settings.
    """
    music_type = "energetic" if energetic else "chill"
    volume = "loud" if loud else "quiet"
    return {"music_type": music_type, "volume": volume}

def dim_lights_impl(brightness: float) -> dict:
    """Dim the lights.

    Args:
        brightness: The brightness of the lights, 0.0 is off, 1.0 is full.

    Returns:
        A dictionary containing the new brightness setting.
    """
    return {"brightness": brightness}

In [127]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
function_declarations = [
    {
        "name": "power_disco_ball",
        "description": "Powers the spinning disco ball.",
        "parameters": {
            "type": "object",
            "properties": {
                "power": {
                    "type": "boolean",
                    "description": "Whether to turn the disco ball on or off.",
                }
            },
            "required": ["power"],
        },
    },
    {
        "name": "start_music",
        "description": "Play some music matching the specified parameters.",
        "parameters": {
            "type": "object",
            "properties": {
                "energetic": {
                    "type": "boolean",
                    "description": "Whether the music is energetic or not.",
                },
                "loud": {
                    "type": "boolean",
                    "description": "Whether the music is loud or not.",
                },
            },
            "required": ["energetic", "loud"],
        },
    },
    {
        "name": "dim_lights",
        "description": "Dim the lights.",
        "parameters": {
            "type": "object",
            "properties": {
                "brightness": {
                    "type": "number",
                    "description": "The brightness of the lights, 0.0 is off, 1.0 is full.",
                }
            },
            "required": ["brightness"],
        },
    }
]

house_tools = types.Tool(function_declarations=function_declarations)
config = types.GenerateContentConfig(
    tools = [house_tools],
    automatic_function_calling=types.AutomaticFunctionCallingConfig(
        disable=True
    ),
    tool_config=types.ToolConfig(
        function_calling_config=types.FunctionCallingConfig(
            mode="ANY",
        )
    ),
)

chat = client.chats.create(
    model='gemini-2.0-flash',
    config=config,
)
response = chat.send_message("Turn this place into a party.")
# Print out each of the function calls requested from this single call
print("Example 1: Forced function calling")
for fn in response.function_calls:
    args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
    print(f"{fn.name}({args})")

Example 1: Forced function calling
power_disco_ball(power=True)
start_music(energetic=True, loud=True)
dim_lights(brightness=0.5)


In [137]:
config = types.GenerateContentConfig(
    tools=[power_disco_ball_impl, start_music_impl, dim_lights_impl]
)

# Make the request
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Do everything you need to this place into party!",
    config=config,
)

print("\nExample 2: Automatic function calling")
print(response.text)


Example 2: Automatic function calling
Alright, I've got the disco ball spinning, the music is loud and energetic, and the lights are dimmed. The party is officially started!
